In [46]:
import pyvo as vo
import pandas as pd
import numpy as np
import os
from astropy.time import Time
from astropy import units as u
from tqdm import tqdm, trange
from math import ceil
from astropy.time import TimeDelta
import time
import threading
import multiprocessing
import glob
import pandas as pd

In [22]:
def get_id(start,end):
    tap_service = vo.dal.TAPService("https://api.skymapper.nci.org.au/public/tap/")   
    query = f"""SELECT object_id, ngood FROM dr2.master where object_id >= {start} and object_id < {end}"""
    tap_results = tap_service.search(query)
    df = pd.DataFrame(tap_results) 
    DATA_DIR = "/epyc/projects/adam_datasets/skyMapper_dr2/master"
    file_name = os.path.join(DATA_DIR, f"obj_id_{start}-{end}.csv")
    df.to_csv(path_or_buf= file_name)

In [23]:
def group_get(start,end):
    for i in range(start,end):
        get_id(i * 1000000, (i+1) * 1000000)

In [ ]:
#56730,58192
jobs = []
Slice = ceil ((506) / 20)
for i in range(20):
	process = multiprocessing.Process(target=group_get, args=(Slice * i, Slice * (i+1)))
	jobs.append(process)

for j in jobs: 
	j.start()

for j in jobs:
	j.join()


In [20]:
store = pd.read_csv("/epyc/projects/adam_datasets/skyMapper_dr2/master/obj_id_15000000-16000000.csv")

In [21]:
store

,Unnamed: 0,object_id
0,0,15623401
1,1,15433269
2,2,15490934
3,3,15417910
4,4,15456338
...,...,...
399112,399112,15695293
399113,399113,15695296
399114,399114,15695297
399115,399115,15695301


In [27]:
path = "/epyc/projects/adam_datasets/skyMapper_dr2/master/"

all_files = glob.glob(os.path.join(path, "obj_id_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "/epyc/projects/adam_datasets/skyMapper_dr2/master/merged.csv")

In [28]:
store = pd.read_csv("/epyc/projects/adam_datasets/skyMapper_dr2/master/merged.csv")

In [30]:
store = store.loc[:, ~store.columns.str.contains('^Unnamed')]

In [32]:
store = store[store['ngood'] <= 4]

In [35]:
store.dtypes

ngood        float64
object_id    float64
dtype: object

In [36]:
store = store.astype({"object_id": int}, errors='raise') 

In [37]:
store.dtypes

ngood        float64
object_id      int64
dtype: object

In [40]:
store = store.astype({"object_id": str}, errors='raise') 

In [60]:
def to_csv(start,end,filter):
    for index in range(start,end):
        date = Time(index, format='mjd') #Timeobject
        date = date.to_value('iso',subfmt='date') #String formate 2014-03-20
        DATA_DIR = "/epyc/projects/adam_datasets/skyMapper_dr2/data_csv"
        file_name = os.path.join(DATA_DIR, f"dr2_observations_{date}.csv")
        if(not os.path.isfile(file_name)):
            continue

        store = pd.read_csv(file_name)
        store = store[store.obj_id.isin(fliter.object_id)]
        store = store.loc[:, ~store.columns.str.contains('^Unnamed')]
        DATA_DIR = "/epyc/projects/adam_datasets/skyMapper_dr2/data_csv_filter"
        file_name = os.path.join(DATA_DIR, f"dr2_observations_{date}.csv")
        store.to_csv(path_or_buf= file_name, index = False)

In [ ]:
filter = pd.read_csv("/epyc/projects/adam_datasets/skyMapper_dr2/master/filter.csv")
#56730,58192
jobs = []
Slice = ceil ((58192-56730) / 20)
for i in range(20):
	process = multiprocessing.Process(target=to_csv, args=(56730+Slice * i, 56730+Slice * (i+1), filter ))
	jobs.append(process)

for j in jobs: 
	j.start()

for j in jobs:
	j.join()

In [ ]:
store.to_csv("/epyc/projects/adam_datasets/skyMapper_dr2/master/filter.csv")

In [53]:
store = pd.read_csv("/epyc/projects/adam_datasets/skyMapper_dr2/data_csv/dr2_observations_2015-05-23.csv")

In [49]:
fliter

,Unnamed: 0,ngood,object_id
0,0,1.0,26517638
1,6,2.0,26458518
2,16,4.0,26458529
3,17,4.0,26458530
4,18,3.0,26458548
...,...,...,...
227407035,505176640,4.0,285795496
227407036,505176648,4.0,285795504
227407037,505176650,4.0,285795511
227407038,505176652,4.0,285795513


In [54]:
store = store[store.obj_id.isin(fliter.object_id)]

In [55]:
store

,Unnamed: 0,dec,dec_sigma,exposure_id,filter,mag,mag_sigma,mjd,obj_id,observatory_code,ra,ra_sigma,obs_id,exposure_mjd_start,exposure_duration,exposure_mjd_mid
12,12,-6.962363,0.000007,20150523120757,r,20.316904,0.0170,57165.506134,109879042,Q55,209.689481,0.000012,1098790421220150523120757,57165.505556,100.0,57165.506134
14,14,-6.936335,0.000009,20150523120757,r,20.014769,0.0301,57165.506134,109879046,Q55,209.666721,0.000009,1098790461220150523120757,57165.505556,100.0,57165.506134
23,23,-6.899634,0.000012,20150523120757,r,19.894920,0.0962,57165.506134,109879774,Q55,209.845459,0.000010,1098797741220150523120757,57165.505556,100.0,57165.506134
24,24,-6.765693,0.000009,20150523120757,r,20.024810,0.0071,57165.506134,109880092,Q55,209.700518,0.000008,1098800921220150523120757,57165.505556,100.0,57165.506134
55,55,-6.733101,0.000089,20150523120757,r,20.603783,0.0613,57165.506134,109882816,Q55,210.102624,0.000074,1098828161620150523120757,57165.505556,100.0,57165.506134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5669493,5669494,0.012163,0.000010,20150523184241,i,19.750536,0.0278,57165.780243,305616808,Q55,304.157366,0.000010,3056168083220150523184241,57165.779664,100.0,57165.780243
5669494,5669495,0.020951,0.000027,20150523184241,i,19.875895,0.0408,57165.780243,305616807,Q55,304.149589,0.000013,3056168073220150523184241,57165.779664,100.0,57165.780243
5669495,5669496,0.008272,0.000022,20150523184241,i,19.596750,0.0246,57165.780243,305616806,Q55,304.159261,0.000052,3056168063220150523184241,57165.779664,100.0,57165.780243
5669497,5669498,0.007888,0.000010,20150523184241,i,19.844635,0.0222,57165.780243,305616803,Q55,304.152008,0.000010,3056168033220150523184241,57165.779664,100.0,57165.780243


In [ ]:
index = None

In [ ]:
python /epyc/ssd/users/yyang35/precovery/precovery/ingest/index_observations.py data_csv_filter database_filter SKYMAPPER_DR2 --nside 32 --cpu_count 20 --dataset_name "SkyMapper Southern Sky Survey (DR2)" --reference_doi https://doi.org/10.3847/1538-3881/abd6e1 --documentation_url https://skymapper.anu.edu.au/data-release/dr2/ --sia_url https://api.skymapper.nci.org.au/aus/siap/dr2/query?